<a href="https://colab.research.google.com/github/PranjalSahu/OAI_analysis_2/blob/pranjal/DaskComputationCoiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages

# Remember to Restart runtime after installation


!pip install itk==5.3rc4
!pip install vtk
!pip install itkwidgets
!pip install icon-registration==0.3.4
#!pip install "dask[complete]"
#!pip install coiled dask distributed
!pip install tornado==6.1
!pip install coiled
!pip install torch
!pip install jupyter
!pip install girder_client
!pip install git+https://github.com/uncbiag/mermaid.git
!pip install git+https://github.com/uncbiag/easyreg.git
!pip install git+https://github.com/PranjalSahu/OAI_analysis_2.git#egg=oai_package

In [1]:
# All Imports

import numpy as np
import itk
import vtk
import itkwidgets
import icon_registration
import icon_registration.itk_wrapper as itk_wrapper
import icon_registration.pretrained_models as pretrained_models
from oai_analysis_2 import mesh_processing as mp

import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import boto3
import coiled
import dask
from dask import delayed, compute, visualize
from dask.distributed import Client, progress, LocalCluster

In [2]:
# Create the environment

if False:
  coiled.create_software_environment(
    name="oai5",
    pip="coiled_requirements",
  )
  # coiled.create_software_environment(
  #   name="oai4",
  #   container="pranjalsahu/pranjal-sahu-oai4:cf409eeb-435c-4976-afd4-6a5118d6b683",
  # )

In [3]:
# Create Dask Client. This will also spawn dask worker and scheduler

use_coiled = False

if use_coiled:
  name = None
  cluster = coiled.Cluster(n_workers=2,
                         worker_cpu=8,
                         worker_memory='31G',
                         name=name,
                         software='pranjal-sahu/oai5')
  client = dask.distributed.Client(cluster, 
                                   serializers=['pickle', 'dask'],
                                   deserializers=['pickle', 'dask'])
  client
else:
  cluster = LocalCluster(processes=False)
  client = Client(cluster)
  client

In [4]:
cluster

In [5]:
# For fetching files from S3

import boto3
s3 = boto3.resource("s3")

def show_contents_s3_bucket(bucket_name):
  import boto3

  s3     = boto3.resource("s3")
  bucket = s3.Bucket(bucket_name)

  all_objects = []
  for obj in bucket.objects.all():
    all_objects.append(str(obj.key))
  
  return ",".join(all_objects)

#print(f'filename : {obj.key} ')
#s3.Bucket(bucket_name).download_file(obj.key, './OAIData1/'+obj.key)
    
bucket_name = 'oaisample1'
show_contents_s3_bucket(bucket_name)

NoCredentialsError: ignored

In [ ]:
# Read File from S3 from Dask workers inside the EC2 instances
# Now finish the pipeline

@delayed
def show_contents_s3_bucket_a(bucket_name, filename):
  import boto3
  import itk

  s3     = boto3.resource("s3")
  bucket = s3.Bucket(bucket_name)

  #all_objects = []
  #for obj in bucket.objects.all():
  #  all_objects.append(str(obj.key))
  #all_files = ",".join(all_objects)

  s3.Bucket(bucket_name).download_file(filename, filename)
  itk_image = itk.imread(filename)
  return itk_image.shape

bucket_name = 'oaisample1'
filename    = 'atlas_image.nii.gz'
output_str  = show_contents_s3_bucket_a(bucket_name, filename)

l = compute(output_str)
print(l)

In [ ]:
cluster

In [ ]:
#client.close()
#cluster.close()

In [6]:
# Download the relevant files such as input image, atlas image, segmented TC and FC maps

!girder-client --api-url https://data.kitware.com/api/v1 localsync 621a5d784acac99f426c143b OAIData

Local metadata does not exists. Falling back to download.
[####################################]  74.74M/74.74M  100%  atlas_image.nii.gz            
[####################################]  76.70M/76.70M  100%  FC_probmap.nii.gz            
[####################################]  27.96M/27.96M  100%  image_preprocessed.nii.gz            
[####################################]  76.66M/76.66M  100%  TC_probmap.nii.gz            


In [7]:
# All Function Definitions with Dask Delayed Decorator to perform parallel Computing

#@delayed(nout=2)
def read_images(image_preprocessed, atlas_image):
    image_A = itk.imread(image_preprocessed, itk.D)
    image_B = itk.imread(atlas_image, itk.D)

    fc_prob_file = './OAIData/FC_probmap.nii.gz'
    FC_prob = itk.imread(fc_prob_file)

    tc_prob_file = './OAIData/TC_probmap.nii.gz'
    TC_prob = itk.imread(tc_prob_file)

    return image_A, image_B, FC_prob, TC_prob

#@delayed(nout=3)
def register_images_delayed():
    import boto3
    import itk
    import icon_registration
    import icon_registration.itk_wrapper as itk_wrapper
    import icon_registration.pretrained_models as pretrained_models

    #s3          = boto3.resource("s3")
    #bucket_name = 'oaisample1'
    #bucket      = s3.Bucket(bucket_name)

    image_A = './OAIData/image_preprocessed.nii.gz'
    image_B = './OAIData/atlas_image.nii.gz'

    #s3.Bucket(bucket_name).download_file(image_A, image_A)
    image_A = itk.imread(image_A, itk.D)

    #s3.Bucket(bucket_name).download_file(image_B, image_B)
    image_B = itk.imread(image_B, itk.D)

    model = pretrained_models.OAI_knees_gradICON_model()
    model.to('cpu')

    # Register the images
    phi_AB, phi_BA = itk_wrapper.register_pair(model, image_A, image_B)
    return phi_AB, itk.dict_from_image(image_A), itk.dict_from_image(image_B)

#@delayed
def deform_probmap_FC_delayed(phi_AB, image_A, image_B):
    import itk
    import boto3

    #s3          = boto3.resource("s3")
    #bucket_name = 'oaisample1'
    #bucket      = s3.Bucket(bucket_name)

    #phi_AB  = itk.transform_from_dict(phi_AB)
    image_A = itk.image_from_dict(image_A)
    image_B = itk.image_from_dict(image_B)

    fc_prob_file = './OAIData/FC_probmap.nii.gz'
    #s3.Bucket(bucket_name).download_file(fc_prob_file, fc_prob_file)
    FC_prob = itk.imread(fc_prob_file, itk.D)

    #print('Image is ')
    #print(image_A)

    #print('FC_prob image is ')
    #print(FC_prob)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    
    #print('Interpolator is ')
    #print(interpolator)

    warped_image = itk.resample_image_filter(FC_prob, 
        transform=phi_AB, 
        interpolator=interpolator,
        size=itk.size(image_B),
        output_spacing=itk.spacing(image_B),
        output_direction=image_B.GetDirection(),
        output_origin=image_B.GetOrigin()
    )

    return itk.dict_from_image(warped_image)
  
#@delayed
def deform_probmap_TC_delayed(phi_AB, image_A, image_B):
    import itk
    import boto3

    #s3          = boto3.resource("s3")
    #bucket_name = 'oaisample1'
    #bucket      = s3.Bucket(bucket_name)

    phi_AB  = itk.transform_from_dict(phi_AB)
    image_A = itk.image_from_dict(image_A)
    image_B = itk.image_from_dict(image_B)

    fc_prob_file = './OAIData/TC_probmap.nii.gz'
    #s3.Bucket(bucket_name).download_file(fc_prob_file, fc_prob_file)
    FC_prob = itk.imread(fc_prob_file, itk.D)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    warped_image = itk.resample_image_filter(FC_prob, 
        transform=phi_AB, 
        interpolator=interpolator,
        size=itk.size(image_B),
        output_spacing=itk.spacing(image_B),
        output_direction=image_B.GetDirection(),
        output_origin=image_B.GetOrigin()
    )

    return itk.dict_from_image(warped_image)

    #return image_A.shape
    #return warped_image_FC

@delayed
def deform_probmap_FC_paramA(phi_AB):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk
    #return itk.dict_from_transform(phi_AB)["transformName"]#.GetNumberOfParameters()
    #return warped_image_FC
    return phi_AB["transformName"]

@delayed
def deform_probmap_TC(phi_AB, image_A, image_B, TC_prob):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk
    #tc_prob_file = './OAIData/TC_probmap.nii.gz'
    #TC_prob = itk.imread(tc_prob_file)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    warped_image_TC = itk.resample_image_filter(TC_prob, 
        transform=phi_AB, 
        interpolator=interpolator,
        size=itk.size(image_B),
        output_spacing=itk.spacing(image_B),
        output_direction=image_B.GetDirection(),
        output_origin=image_B.GetOrigin()
    )

    return warped_image_TC

# Convert from VTK Mesh to ITK Mesh to make it serializable
def get_itk_mesh(vtk_mesh):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk

    Dimension = 3
    PixelType = itk.D
    
    MeshType = itk.Mesh[PixelType, Dimension]
    itk_mesh = MeshType.New()
    
    # Get points array from VTK mesh
    points = vtk_mesh.GetPoints().GetData()
    points_numpy = np.array(points).flatten().astype('float32')
        
    polys = vtk_mesh.GetPolys().GetData()
    polys_numpy = np.array(polys).flatten()

    # Triangle Mesh
    vtk_cells_count = vtk_mesh.GetNumberOfPolys()
    polys_numpy = np.reshape(polys_numpy, [vtk_cells_count, Dimension+1])

    # Extracting only the points by removing first column that denotes the VTK cell type
    polys_numpy = polys_numpy[:, 1:]
    polys_numpy = polys_numpy.flatten().astype(np.uint64)

    # Get point data from VTK mesh to insert in ITK Mesh
    point_data_numpy = np.array(vtk_mesh.GetPointData().GetScalars())#.astype('float64')
    
    # Get cell data from VTK mesh to insert in ITK Mesh
    cell_data_numpy = np.array(vtk_mesh.GetCellData().GetScalars())#.astype('float64')
    
    itk_mesh.SetPoints(itk.vector_container_from_array(points_numpy))
    itk_mesh.SetCellsArray(itk.vector_container_from_array(polys_numpy), itk.CommonEnums.CellGeometry_TRIANGLE_CELL)
    itk_mesh.SetPointData(itk.vector_container_from_array(point_data_numpy))
    itk_mesh.SetCellData(itk.vector_container_from_array(cell_data_numpy))    
    return itk_mesh
    
@delayed
def get_thickness_FC(warped_image_FC):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk

    distance_inner_FC, distance_outer_FC = mp.get_thickness_mesh(warped_image_FC, mesh_type='FC')
    distance_inner_FC = get_itk_mesh(distance_inner_FC)
    return distance_inner_FC

@delayed
def get_thickness_TC(warped_image_TC):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk

    distance_inner_TC, distance_outer_TC = mp.get_thickness_mesh(warped_image_TC, mesh_type='TC')
    distance_inner_TC = get_itk_mesh(distance_inner_TC)
    return distance_inner_TC

#@delayed(nout=2)
def get_paths():
    import numpy as np
    import itk
    import vtk
    import itkwidgets
    import icon_registration
    import icon_registration.itk_wrapper as itk_wrapper
    import icon_registration.pretrained_models as pretrained_models
    from oai_analysis_2 import mesh_processing as mp

    image_preprocessed = "./OAIData/image_preprocessed.nii.gz"
    atlas_image = "./OAIData/atlas_image.nii.gz"

    return image_preprocessed, atlas_image

#@delayed
def only_register_images(image_A, image_B):
  #import itkConfig
  #itkConfig.LazyLoading = False
  #import itk
  import numpy as np
  import itk
  import vtk
  import itkwidgets
  import icon_registration
  import icon_registration.itk_wrapper as itk_wrapper
  import icon_registration.pretrained_models as pretrained_models
  from oai_analysis_2 import mesh_processing as mp

  phi_AB  = register_images(image_A=image_A, image_B=image_B)
  return phi_AB

@delayed
def import_all():
  import numpy as np
  #import itkConfig
  #itkConfig.LazyLoading = False
  import itk
  import vtk
  import itkwidgets
  import icon_registration
  import icon_registration.itk_wrapper as itk_wrapper
  import icon_registration.pretrained_models as pretrained_models
  from oai_analysis_2 import mesh_processing as mp

  return True

In [8]:
phi_AB, image_A, image_B = register_images_delayed()
#deformed_fc = deform_probmap_FC_delayed(phi_AB, image_A, image_B)
#deformed_tc = deform_probmap_TC_delayed(phi_AB, image_A, image_B)

#result = [deformed_fc, deformed_tc]
#output_result = compute(*result)
#print(output_result)

In [21]:
td1 = itk.transform_from_dict(d1)
print(td1)

CompositeTransform (0x83070c0)
  RTTI typeinfo:   itk::CompositeTransform<double, 3u>
  Reference Count: 1
  Modified Time: 1042
  Debug: Off
  Object Name: 
  Observers: 
    none
  Transforms in queue, from begin to end:
  >>>>>>>>>
  CenteredAffineTransform (0xc361c80)
    RTTI typeinfo:   itk::CenteredAffineTransform<double, 3u>
    Reference Count: 1
    Modified Time: 1033
    Debug: Off
    Object Name: 
    Observers: 
      none
    Matrix: 
      1 0 0 
      0 1 0 
      0 0 1 
    Offset: [0, 0, 0]
    Center: [0, 0, 0]
    Translation: [0, 0, 0]
    Inverse: 
      1 0 0 
      0 1 0 
      0 0 1 
    Singular: 0
  >>>>>>>>>
  DisplacementFieldTransform (0x7b413a0)
    RTTI typeinfo:   itk::DisplacementFieldTransform<double, 3u>
    Reference Count: 1
    Modified Time: 1035
    Debug: Off
    Object Name: 
    Observers: 
      none
    DisplacementField: (null)
    InverseDisplacementField: (null)
    Interpolator: 
      VectorLinearInterpolateImageFunction (0x8fb0580)


In [27]:
print(phi_AB1.GetNthTransform(1)["parameters"])

[]


In [32]:
temp1 = phi_AB1.GetNthTransform(1)
print(temp1["parameters"])
temp1["parameters"] = phi_AB.GetNthTransform(1)["parameters"]+1
print(temp1["parameters"])

[-1.55766129 -1.21581364 -0.17686807 ...  0.61738586  2.51548004
  9.465065  ]
[-0.55766129 -0.21581364  0.82313193 ...  1.61738586  3.51548004
 10.465065  ]


In [34]:
itk.dict_from_transform(phi_AB1)

[{'fixedParameters': array([], dtype=float64),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 0,
  'numberOfParameters': 0,
  'numberOfTransforms': 3,
  'outDimension': 3,
  'parameters': array([], dtype=float64),
  'parametersValueType': 'float64',
  'transformName': 'CompositeTransform',
  'transformType': 'D3'},
 {'fixedParameters': array([0., 0., 0.]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 3,
  'numberOfParameters': 15,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
  'parametersValueType': 'float64',
  'transformName': 'CenteredAffineTransform',
  'transformType': 'D33'},
 {'fixedParameters': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 18,
  'numberOfParameters': 8847360,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([-0.5576

In [33]:
itk.dict_from_transform(phi_AB)

[{'fixedParameters': array([], dtype=float64),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 0,
  'numberOfParameters': 0,
  'numberOfTransforms': 3,
  'outDimension': 3,
  'parameters': array([], dtype=float64),
  'parametersValueType': 'float64',
  'transformName': 'CompositeTransform',
  'transformType': 'D3'},
 {'fixedParameters': array([95.5, 95.5, 39.5]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 3,
  'numberOfParameters': 15,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([   0.        ,    0.        ,   -1.40000153,    0.72916669,
            0.        ,    0.        ,    0.        ,   -0.72916669,
            0.        ,   95.5       ,   95.5       ,   39.5       ,
         -151.15006065,  -25.68228976, -109.31771024]),
  'parametersValueType': 'float64',
  'transformName': 'CenteredAffineTransform',
  'transformType': 'D33'},
 {'fixedParameters': array([192., 192.,  80.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,   0

In [26]:
print(phi_AB.GetNthTransform(1)["parameters"])

[-1.55766129 -1.21581364 -0.17686807 ...  0.61738586  2.51548004
  9.465065  ]


In [ ]:
deformed_fc = deform_probmap_FC_delayed(phi_AB, image_A, image_B)

In [10]:
phi_AB1 = itk.transform_from_dict(d1)

In [13]:
phi_AB1["fixedParameters"]
print(phi_AB["fixedParameters"])

[]


In [11]:
print(phi_AB1)

CompositeTransform (0x8307c20)
  RTTI typeinfo:   itk::CompositeTransform<double, 3u>
  Reference Count: 1
  Modified Time: 1031
  Debug: Off
  Object Name: 
  Observers: 
    none
  Transforms in queue, from begin to end:
  >>>>>>>>>
  CenteredAffineTransform (0xc363900)
    RTTI typeinfo:   itk::CenteredAffineTransform<double, 3u>
    Reference Count: 1
    Modified Time: 1022
    Debug: Off
    Object Name: 
    Observers: 
      none
    Matrix: 
      1 0 0 
      0 1 0 
      0 0 1 
    Offset: [0, 0, 0]
    Center: [0, 0, 0]
    Translation: [0, 0, 0]
    Inverse: 
      1 0 0 
      0 1 0 
      0 0 1 
    Singular: 0
  >>>>>>>>>
  DisplacementFieldTransform (0x7b405a0)
    RTTI typeinfo:   itk::DisplacementFieldTransform<double, 3u>
    Reference Count: 1
    Modified Time: 1024
    Debug: Off
    Object Name: 
    Observers: 
      none
    DisplacementField: (null)
    InverseDisplacementField: (null)
    Interpolator: 
      VectorLinearInterpolateImageFunction (0x3601f360)

In [9]:
d1 = itk.dict_from_transform(phi_AB)

In [ ]:
print(phi_AB)

In [ ]:
d1

In [ ]:
d1 = itk.dict_from_transform(phi_AB)
phi_AB1 = itk.transform_from_dict(d1)

print(phi_AB1)

In [ ]:
phi_AB1_fixed_params = phi_AB1.GetNthTransform(0)#['fixedParameters']
print(phi_AB1_fixed_params["fixedParameters"])

phi_AB_fixed_params = phi_AB.GetNthTransform(0)#['fixedParameters']
print(phi_AB_fixed_params["fixedParameters"])

#print(phi_AB1_fixed_params.GetFixedParameters())

In [1]:
import itk
import numpy as np
import pickle

Dimension = 3
PixelType = itk.D

# List of Transforms to test
transforms_to_test = [itk.DisplacementFieldTransform[PixelType, Dimension]]#[itk.AffineTransform[PixelType, Dimension], itk.DisplacementFieldTransform[PixelType, Dimension], itk.BSplineTransform[PixelType, Dimension, 3], itk.QuaternionRigidTransform[PixelType]]

keys_to_test1 = ["name", "parametersValueType", "transformName", "transformType", "inDimension", "outDimension", "numberOfParameters", "numberOfFixedParameters"]
keys_to_test2 = ["parameters", "fixedParameters"]

transform_object_list = []
for i, transform_type in enumerate(transforms_to_test):
    print(transform_type)

    transform = transform_type.New()
    transform.SetObjectName("transform"+str(i))
    print(transform.GetObjectName())

    fixedParameters = np.random.rand(3, 3, 2).flatten()
    print(transform.GetFixedParameters().Size())

    #print(fixedParameters.dtype)
    transform["fixedParameters"] = fixedParameters

    #print(transform["fixedParameters"])

    # Check the serialization
    serialize_deserialize = pickle.loads(pickle.dumps(transform))

    #print(serialize_deserialize)
    # # Test all the attributes
    for k in keys_to_test1:
        assert serialize_deserialize[k] == transform[k]

    # # Test all the parameters
    for k in keys_to_test2:
        print(k)
        print(serialize_deserialize[k])
        print(transform[k])
        assert np.array_equal(serialize_deserialize[k], transform[k])

    transform.SetFixedParameters(transform.GetFixedParameters())
    #transform["fixedParameters"] = fixedParameters#transform.GetFixedParameters()
    #print(transform)
    # transform_object_list.append(transform)

print('Individual Transforms Test Done')

ModuleNotFoundError: ignored

In [ ]:
transform.SetFixedParameters

In [ ]:
t1 = transform_object_list[1]
print(t1["fixedParameters"])

In [ ]:
transform_template = getattr(itk, 'DisplacementFieldTransform')
transform_template

In [ ]:
a = np.array([3, 3])
print(a.shape)
print(np.random.rand(*a))

In [ ]:
output_result = compute(result)
print(output_result)

In [ ]:
img_np = np.random.rand(5, 5)
print(img_np.shape)

img = itk.image_from_array(img_np)
print(img.shape)

In [ ]:
# Create a test mesh

m = itk.Mesh.F3.New()
p = np.random.rand(5, 3).astype('float32')
print(p.shape, p.dtype)
m.SetPoints(itk.vector_container_from_array(p.flatten()))

#md = itk.dict_from_mesh(m)
#print(md)

In [ ]:
@delayed
def perform_sum1(img_temp):
  import itk
  mesh = itk.mesh_from_dict(img_temp)
  return img_temp['dimension']

  #return np.sum(img_temp['points'])
  #return itk.di
  #import itkConfig
  #itkConfig.LazyLoading = False
  #import itk
  #return itk.image_from_array(img_temp)
  #a = np.array(img_temp)
  #return a#np.sum(img_temp)

In [ ]:
# Create Dask Computation Graph

result = []

#import_done = import_all()
image_preprocessed, atlas_image   = get_paths()

image_A, image_B, FC_prob, TC_prob  = read_images(image_preprocessed, atlas_image)
phi_AB            = only_register_images(image_A, image_B)

warped_image_FC = deform_probmap_FC(phi_AB, image_A, image_B, FC_prob)
warped_image_TC = deform_probmap_TC(phi_AB, image_A, image_B, TC_prob)

thickness_FC = get_thickness_FC(warped_image_FC)
thickness_TC = get_thickness_TC(warped_image_TC)

result.append(thickness_FC)
result.append(thickness_TC)

In [ ]:
numParam = deform_probmap_FC_paramA(phi_AB)


In [ ]:
m1 = itk.dict_from_mesh(m)
sumImg = perform_sum1(m1)

In [ ]:
#cluster.close()

In [ ]:
# Visualize Dask Computation Graph

visualize(sumImg)

In [ ]:
# Perform computation
#%%time
l = compute(sumImg)

In [ ]:
print(l)

In [ ]:
import pickle
import cloudpickle
a = itk.image_from_array(img_np)
b = cloudpickle.loads(pickle.dumps(a))
print(b)

In [ ]:
!pip install cloudpickle

![DaskProgress](https://user-images.githubusercontent.com/1044135/163692457-40a41395-3d83-4928-9445-528cd025a7b9.png)


In [ ]:
# Write the result meshes

distance_inner_FC = l[0]
distance_inner_TC = l[1]

itk.meshwrite(distance_inner_FC, 'itk_distance_inner_FC.vtk')
itk.meshwrite(distance_inner_TC, 'itk_distance_inner_TC.vtk')

In [ ]:
# Visualize the meshes with thickness as cell data

# To enable running the itkwidgets window on colab
from google.colab import output
output.enable_custom_widget_manager()

a1 = vtk.vtkPolyDataReader()
a1.SetFileName('itk_distance_inner_FC.vtk')
a1.Update()
distance_inner_FC = a1.GetOutput()

itkwidgets.view(geometries=[distance_inner_FC])